In [ ]:
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from comb.behavior_ophys_dataset import BehaviorOphysDataset

def read_DLC_h5file(h5file):
    """
    Read in a h5 file from DLC and return a dataframe with the DLC data wiht clean column names

    Parameters
    ----------
    h5file : str
        full path to the h5 file"""
    df = pd.read_hdf(h5file, key='df_with_missing')
    junk = df.keys()[0][0]  # this is a hack to get the right column names
    df = df[junk]
    df_dlc = pd.DataFrame(df.unstack())  # long format shape
    df_dlc = df_dlc.reset_index()
    df_dlc = df_dlc.rename(columns={0: 'value', 'level_2': 'frame_number'})  # rename columns to clear names
    print('loaded DLC data')
    return df_dlc

In [ ]:
plane_path = '/root/capsule/data/multiplane-ophys_736963_2024-07-09_09-13-58_processed_2024-11-20_19-03-42/VISp_0'
raw_path = '/root/capsule/data/multiplane-ophys_736963_2024-07-09_09-13-58'
eye_path = '/root/capsule/data/multiplane-ophys_736963_2024-07-09_09-13-58_dlc-eye_2025-01-18_00-27-35'
bod = BehaviorOphysDataset(plane_folder_path=plane_path,
                               raw_folder_path=raw_path,
                               eye_tracking_path=eye_path,
                               pipeline_version='v6')

In [ ]:
eye_df = bod.eye_tracking_table.copy(deep=True)
pupil_area_df = eye_df.query('eye_is_bad_frame==False and pupil_is_bad_frame==False')[['timestamps', 'pupil_area']].copy()
fig, ax = plt.subplots()
ax.plot(pupil_area_df['timestamps'], pupil_area_df['pupil_area'])

In [ ]:
eye_tracking_movie = '/root/capsule/data/multiplane-ophys_736963_2024-07-09_09-13-58/behavior-videos/1378956881_Eye_20240709T091346.mp4'
cap_eye = cv2.VideoCapture(eye_tracking_movie)
width = int(cap_eye.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap_eye.get(cv2.CAP_PROP_FRAME_HEIGHT))
length = int(cap_eye.get(cv2.CAP_PROP_FRAME_COUNT))

dlc_h5 = '/root/capsule/data/multiplane-ophys_736963_2024-07-09_09-13-58_dlc-eye_2025-01-18_00-27-35/1378956881_Eye_20240709T091346DLC_resnet50_universal_eye_trackingJul10shuffle1_1030000.h5'
dlc_df = read_DLC_h5file(dlc_h5)
pupil_points_df = dlc_df[(dlc_df.bodyparts.str.contains('pupil'))]


In [ ]:
frame_nums_to_capture = np.arange(19807, 19820)
frames = []
pupil_points = []
for frame_number in frame_nums_to_capture:
    cap_eye.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
    ret, frame = cap_eye.read()
    frames.append(frame)

    x_values = pupil_points_df.query(f'frame_number=={frame_number} and coords=="x"').value.values
    y_values = pupil_points_df.query(f'frame_number=={frame_number} and coords=="y"').value.values    
    pupil_points.append([x_values, y_values])
    

In [ ]:
for i, frame in enumerate(frames):
    fig, ax = plt.subplots()
    ax.imshow(frame)
    ax.scatter(pupil_points[i][0], pupil_points[i][1], color='r')
    ax.set_title(f'Frame {frame_nums_to_capture[i]}')
    plt.show()
